# 🍽️ AccessGen: AI-Powered Meal Planning Assistant

Are you looking to revolutionize your meal planning? Introducing **AccessGen**, an intelligent meal planning application powered by the cutting-edge **Google Gemini AI**!  

This project aims to provide you with a smarter way to manage your nutrition by combining image analysis, personalized meal generation, and verified nutritional data.

### ✨ Features:
- **📸 Image-Based Food Analysis**: Upload a food image and instantly get calorie & macro estimates using Gemini Vision.
- **🧠 Personalized 7-Day Meal Plans**: Custom meal plans based on dietary goals, preferences, and restrictions.
- **🔍 Verified Nutritional Data**: Backed by the **USDA FoodData Central API** for accuracy and reliability.
- **📊 Comprehensive Food Insights**: Explore detailed nutrition facts grounded in USDA data.
- **🌐 Multi-Language Support**: English, Spanish, French, and German support for global users.
- **🛒 Weekly Grocery Lists**: Auto-generated grocery lists from your weekly meal plan.

> This notebook demonstrates AccessGen's core capabilities, including food recognition, calorie estimation, and 7-day plan generation. Future versions may include mobile integration and advanced dashboards.
    
💡 Whether you're looking to lose weight, gain muscle, or maintain a balanced lifestyle, AccessGen tailors nutrition to you with AI precision.

To get started, ensure you have:
- Python 3.9+
- A Google API key with Gemini & Gemini Vision access
- A free USDA FoodData Central API key

📌 GitHub Project: [fmWaithaka/Nutritionist-AI-Agent](https://github.com/fmWaithaka/Nutritionist-AI-Agent)
🎥 YouTube Demo: [Watch AccessGen in Action](https://www.youtube.com/watch?v=EIQyRj1anj8)
📝 Blog Post: [Read the Development Story](https://medium.com/@pr401n/%EF%B8%8Fai-meal-planner-798d11cca8ee)

### 📦 Required Libraries

This section loads all the essential libraries used throughout the notebook:

- **Core Functionality:** File handling, text parsing, randomization.
- **Data Analysis & Visualization:** For structured data manipulation and visual feedback.
- **Image Processing:** Handle and encode images for Gemini Vision.
- **Interactive UI:** Enable user-friendly widgets for image selection and language choice.
- **API Communication:** Used for sending requests to Gemini and USDA APIs.
- **Kaggle Integration:** Securely fetch API keys using `kaggle_secrets`.
- **Notebook Display Utilities:** Render styled output, markdown, and images inline.

These imports are grouped by purpose for clarity and maintainability.


In [75]:
# Core Libraries
import os
import re
import json
import random
import logging
import ipywidgets as widgets


#  Data Handling & Visualization
import pandas as pd
import matplotlib.pyplot as plt

#  Image Processing
from PIL import Image
from base64 import b64encode

#  API & Networking
import requests
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
from typing import Dict, List, Any, Optional

#  Secrets for API Keys (Kaggle)
from kaggle_secrets import UserSecretsClient

#  Jupyter Display Tools
from IPython.display import display, Markdown, HTML, clear_output
from IPython.display import Image as IPyImage

# Logging Config
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('AccessGen')


#### 🔐 API Configuration: Gemini & USDA

This section configures access to the two main APIs used in AccessGen:

---

#### 🔹 **Google Gemini API (for Generative AI)**
Used for generating personalized meal plans, calorie analysis from food images, and multilingual support.

To obtain your Gemini API key:
1. Visit [https://makersuite.google.com/app/apikey](https://makersuite.google.com/app/apikey)
2. Log in with your Google account.
3. Generate and copy the API key.

Store the key as a Kaggle secret with the name: `GOOGLE_API_KEY`.

---

#### 🔹 **USDA FoodData Central API**
Used to ground and validate nutritional data with verified values from the USDA database.

To obtain your USDA API key:
1. Visit [https://fdc.nal.usda.gov/api-key-signup.html](https://fdc.nal.usda.gov/api-key-signup.html)
2. Register to get a free API key.
3. Save it as a Kaggle secret with the name: `USDA_FDC_API`.

---

These API keys are securely accessed using Kaggle’s `UserSecretsClient`.  
This ensures that keys are **never exposed** directly in the notebook or logs.


In [76]:
# Gemini API setup
KAGGLE_SECRET_KEY_NAME = "GOOGLE_API_KEY"
GEMINI_API_KEY = UserSecretsClient().get_secret(KAGGLE_SECRET_KEY_NAME)
GEMINI_MODEL_NAME = "models/gemini-2.0-flash"
GEMINI_API_URL = f"https://generativelanguage.googleapis.com/v1beta/{GEMINI_MODEL_NAME}:generateContent?key={GEMINI_API_KEY}"

# USDA API Setup
USDA_API_KEY = UserSecretsClient().get_secret("USDA_FDC_API")
USDA_API_URL = "https://api.nal.usda.gov/fdc/v1/foods/search"

# Default request headers
DEFAULT_HEADERS = {"Content-Type": "application/json"}

### 🔍 USDA Nutrition Lookup Utilities

This section defines two helper functions that power the USDA grounding features in AccessGen:

---

#### `fetch_usda_nutrition(food_name)`
This function takes a food name or ingredient and:
- Searches the **USDA FoodData Central API** for a matching entry.
- Retrieves detailed nutrient data for **calories, protein, carbs, and fat**.
- Returns a clean dictionary of nutritional values.

🔧 **How It Works**:
- Uses the `food/search` endpoint to get the `fdcId`.
- Then calls `food/{fdcId}` to retrieve detailed nutrient information.
- Gracefully handles cases where the food is not found or the response is malformed.

---

####  `extract_ingredients_from_dish(dish_description)`
This function attempts to **split complex dish names** into possible core ingredients.
- It uses a basic text split strategy (based on commas, "and", "with", etc.).
- Helps isolate ingredients for **per-item USDA validation**.

---

These tools allow us to **ground** AI-generated food descriptions using **verified real-world nutrition data**, improving accuracy and transparency.


In [77]:
def fetch_usda_nutrition(food_name):
    try:
        search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
        params = {
            "api_key": USDA_API_KEY,
            "query": food_name,
            "pageSize": 1
        }
        search_res = requests.get(search_url, params=params).json()
        if not search_res.get("foods"):
            return None
        fdc_id = search_res["foods"][0]["fdcId"]

        # Retrieve details
        detail_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
        details = requests.get(detail_url, params={"api_key": USDA_API_KEY}).json()

        # Build dict safely
        nutrients = {}
        for item in details.get("foodNutrients", []):
            name = item.get("nutrient", {}).get("name", "").lower()
            value = item.get("amount", 0)
            nutrients[name] = value

        return {
            "Calories": nutrients.get("energy", "N/A"),
            "Protein (g)": nutrients.get("protein", "N/A"),
            "Carbohydrates (g)": nutrients.get("carbohydrate, by difference", "N/A"),
            "Fat (g)": nutrients.get("total lipid (fat)", "N/A")
        }

    except Exception as e:
        print("❌ USDA lookup failed:", e)
        return None

def extract_ingredients_from_dish(dish_description):
    """
    Very basic keyword splitter to attempt breaking dishes into core ingredients.
    Future: Replace this with Gemini or spaCy-based entity extraction.
    """
    potential_ingredients = re.split(r",|\band\b|with|on|over", dish_description.lower())
    return [i.strip() for i in potential_ingredients if len(i.strip()) > 2]


# 🌍 Language Selection

Use the dropdown below to choose your preferred output language. All text output will follow this setting.

This function is not fully implemented in the model, so just choose English


In [78]:
def language_selector():
    # 🌍 Supported languages
    languages = {
        "English": "en", "Arabic": "ar", "Turkish": "tr",
        "Spanish": "es", "French": "fr", "German": "de", 
        "Italian": "it", "Hindi": "hi"
    }

    # Language dropdown
    lang_dropdown = widgets.Dropdown(
        options=list(languages.keys()),
        value="English",
        description="🌐 Language:"
    )

    # Button to confirm selection
    confirm_button = widgets.Button(description="✅ Confirm Language")
    lang_output = widgets.Output()

    # Global variable to hold selected language
    chosen_lang = lang_dropdown.value
    lang_code = languages[chosen_lang]

    # Update when button is clicked
    def on_confirm_lang(b):
        nonlocal chosen_lang, lang_code
        chosen_lang = lang_dropdown.value
        lang_code = languages[chosen_lang]
        with lang_output:
            lang_output.clear_output()
            display(Markdown(f"✅ **Language set to:** `{chosen_lang}`"))

    confirm_button.on_click(on_confirm_lang)

    # Display
    display(Markdown("### 🌐 Select Your Language"))
    display(lang_dropdown, confirm_button, lang_output)
    
    return lang_code, chosen_lang



# 🖼️ Select a Food Image

Choose a sample food image from the uploaded dataset to analyze its contents and estimate the calories.


In [79]:
from IPython.display import display, Markdown, Image, clear_output
import os
import random
import ipywidgets as widgets

def image_selector(image_root=None, max_images=9):
    """
    Displays a grid of food images for selection and returns the selected image path.
    
    Args:
        image_root (str): Path to the directory containing food category folders.
                         If None, will use current directory.
        max_images (int): Maximum number of images to display (default: 9)
    
    Returns:
        str: Path to the selected image (None if no selection made)
    """
    # Set default path if not provided
    if image_root is None:
        image_root = os.getcwd()
    
    # Verify the directory exists
    if not os.path.exists(image_root):
        display(Markdown(f"### ❌ Error: Directory not found: `{image_root}`"))
        return None
    
    try:
        # Get category directories
        category_dirs = sorted([
            os.path.join(image_root, d) 
            for d in os.listdir(image_root) 
            if os.path.isdir(os.path.join(image_root, d))
        ])
        
        if not category_dirs:
            display(Markdown(f"### ❌ No subdirectories found in `{image_root}`"))
            return None
            
        # Get sample images (handle cases where directories might be empty)
        sample_paths = []
        for cat in category_dirs[:max_images]:
            try:
                files = [f for f in os.listdir(cat) if not f.startswith('.')]  # Skip hidden files
                if files:
                    sample_paths.append(os.path.join(cat, random.choice(files)))
            except Exception as e:
                print(f"Warning: Could not access {cat}: {str(e)}")
        
        if not sample_paths:
            display(Markdown("### ❌ No valid images found in subdirectories"))
            return None

        # Handle click event
        def on_image_click(path):
            def handler(btn):
                nonlocal selected_image_path
                selected_image_path = path
                clear_output(wait=True)
                display(Markdown(f"### ✅ You selected: `{os.path.basename(path)}`"))
                display(Image.open(path))
            return handler

        # Build grid with images + buttons
        widget_rows = []
        for path in sample_paths:
            try:
                with open(path, "rb") as f:
                    image_bytes = f.read()
                
                img_widget = widgets.Image(
                    value=image_bytes,
                    format='jpg',
                    width=150,
                    height=150
                )
                button = widgets.Button(
                    description=os.path.basename(path)[:20], 
                    layout=widgets.Layout(width="160px")
                )
                button.on_click(on_image_click(path))
                widget_rows.append(widgets.VBox([img_widget, button]))
            except Exception as e:
                print(f"Skipping {path}: {str(e)}")

        if not widget_rows:
            display(Markdown("### ❌ No displayable images found"))
            return None

        # Display image selection grid
        display(Markdown("### 🍽️ Choose a food image to analyze:"))
        grid = widgets.GridBox(
            widget_rows, 
            layout=widgets.Layout(grid_template_columns="repeat(3, 200px)")
        )
        display(grid)
        
        # Initialize selected image path
        selected_image_path = None
        
        return selected_image_path
        
    except Exception as e:
        display(Markdown(f"### ❌ Error: {str(e)}"))
        return None



## 📷  Estimate Calories from a Food Image

Snap or upload a food photo and let AccessGen estimate the calorie and nutrient content using Gemini Vision!


In [80]:
# Load API keys
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
USDA_API_KEY = UserSecretsClient().get_secret("USDA_FDC_API")

# Convert image to base64
def image_to_base64(path):
    with open(path, "rb") as f:
        return b64encode(f.read()).decode("utf-8")

# USDA FoodData Central Lookup
def search_usda_food(food_name, max_results=1):
    search_url = f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={USDA_API_KEY}"
    params = {
        "query": food_name,
        "pageSize": max_results
    }

    try:
        response = requests.get(search_url, params=params)
        if response.status_code != 200:
            print(f"❌ USDA API Error: {response.status_code}")
            return None

        foods = response.json().get("foods", [])
        if not foods:
            print("⚠️ No USDA match found.")
            return None

        food_item = foods[0]
        return {
            "description": food_item.get("description", "N/A"),
            "brand": food_item.get("brandOwner", "Generic"),
            "calories": next((n["value"] for n in food_item.get("foodNutrients", []) if n["nutrientName"] == "Energy"), "N/A")
        }
    except Exception as e:
        print("❌ USDA Lookup Error:", e)
        return None

# Analyze food image with Gemini + USDA validation
def analyze_selected_image(path, language="English"):
    if not path:
        return display(Markdown("⚠️ No image selected."))

    image_b64 = image_to_base64(path)

    prompt = (
        f"Estimate the calories in this food image. "
        f"Reply in JSON format like: {{'food': '...', 'estimated_calories': ..., "
        f"'macros': {{'protein': ..., 'carbs': ..., 'fat': ...}}, "
        f"'micros': {{'fiber': ..., 'sugar': ..., 'sodium': ...}}, "
        f"'portion_grams': 100}}. Respond in {language}."
    )

    headers = {"Content-Type": "application/json"}
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GOOGLE_API_KEY}"
    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt},
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",
                            "data": image_b64
                        }
                    }
                ]
            }
        ]
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code != 200:
        return display(Markdown(f"❌ API Error: {response.status_code}\n{response.text}"))

    text_result = response.json()["candidates"][0]["content"]["parts"][0]["text"]

    match = re.search(r"\{.*\}", text_result, re.DOTALL)
    if not match:
        return display(Markdown("⚠️ Could not parse JSON from Gemini.\n\n```json\n" + text_result + "\n```"))

    try:
        result = json.loads(match.group())

        food = result.get("food", "Unknown")
        calories = float(result.get("estimated_calories", 0))
        macros = result.get("macros", {})
        micros = result.get("micros", {})
        portion = result.get("portion_grams", 100)
        burn_minutes = int(calories / 10)

        display(Markdown(f"### 🍽️ Food Detected: **{food}**"))
        display(Markdown(f"🔥 **Calories (Gemini):** `{int(calories)} kcal`"))
        display(Markdown(f"🏃 **Estimated Burn Time (running):** ~`{burn_minutes} min`"))
        display(Markdown(f"📏 **Portion Size (estimated):** `{portion} g`"))

        # USDA lookup
        usda_data = search_usda_food(food)
        if usda_data:
            display(Markdown(f"### 📦 USDA Verified Nutrition"))
            display(Markdown(f"- **Match:** {usda_data['description']}"))
            display(Markdown(f"- **Calories (USDA):** `{usda_data['calories']} kcal`"))

        # Nutrient Table
        nutrient_data = {
            "Nutrient": [
                "Calories", "Protein (g)", "Carbohydrates (g)",
                "Fat (g)", "Fiber (g)", "Sugars (g)", "Sodium (mg)"
            ],
            "Value": [
                f"{int(calories)} kcal",
                f"{int(macros.get('protein', 0))} g",
                f"{int(macros.get('carbs', 0))} g",
                f"{int(macros.get('fat', 0))} g",
                f"{int(micros.get('fiber', 0))} g",
                f"{int(micros.get('sugar', 0))} g",
                f"{int(micros.get('sodium', 0))} mg"
            ]
        }

        df = pd.DataFrame(nutrient_data)

        styled_table = df.style.set_table_styles([
            {'selector': 'thead', 'props': [('background-color', '#222'), ('color', 'black'), ('font-weight', 'bold')]},
            {'selector': 'tbody', 'props': [('background-color', '#111'), ('color', 'black')]},
            {'selector': 'th, td', 'props': [('border', '1px solid #444'), ('padding', '8px')]}
        ]).set_properties(**{'text-align': 'left'}).hide(axis="index")

        display(Markdown("### 🧪 Nutritional Breakdown per Portion"))
        display(styled_table)

    except Exception as e:
        display(Markdown("❌ Parsing failed."))
        print(e)
        print("🔎 Raw output:\n```json\n" + text_result + "\n```")




## 🧾  Tell AccessGen About Your Dietary Goals

To create a truly personalized experience, AccessGen needs to understand your dietary preferences, health goals, and any restrictions.  
Please answer the following questions to help us tailor your meal plan.


In [81]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

def collect_user_preferences():
    """
    Displays a form to collect user dietary preferences and returns them as a dictionary.
    
    Returns:
        dict: Dictionary containing user preferences with keys:
              - goal
              - diet
              - allergies
              - favorite_foods
              - disliked_foods
    """
    # Define widgets
    goal_widget = widgets.Dropdown(
        options=["lose", "maintain", "gain weight"],
        description="Goal:",
        style={'description_width': 'initial'}
    )
    diet_widget = widgets.Text(
        description="Dietary restrictions:",
        style={'description_width': 'initial'}
    )
    allergies_widget = widgets.Text(
        description="Allergies:",
        style={'description_width': 'initial'}
    )
    favorites_widget = widgets.Text(
        description="Favorite foods:",
        style={'description_width': 'initial'}
    )
    disliked_widget = widgets.Text(
        description="Disliked foods:",
        style={'description_width': 'initial'}
    )
    submit_button = widgets.Button(
        description="✅ Submit Preferences", 
        button_style='success',
        layout=widgets.Layout(width='auto')
    )

    # Create output area
    output = widgets.Output()
    
    # Display form
    display(Markdown("### 📝 Fill out your preferences below:"))
    display(widgets.VBox([
        goal_widget,
        diet_widget,
        allergies_widget,
        favorites_widget,
        disliked_widget,
        submit_button,
        output
    ]))

    # Initialize preferences dictionary
    preferences = {}

    # Submit handler
    def on_submit_clicked(b):
        nonlocal preferences
        with output:
            clear_output()
            preferences = {
                "goal": goal_widget.value,
                "diet": diet_widget.value,
                "allergies": allergies_widget.value,
                "favorite_foods": favorites_widget.value,
                "disliked_foods": disliked_widget.value,
            }
            
            # Display confirmation
            display(Markdown("### ✅ Your Preferences"))
            for k, v in preferences.items():
                display(Markdown(f"- **{k.replace('_', ' ').capitalize()}**: {v if v else 'Not specified'}"))

    submit_button.on_click(on_submit_clicked)
    
    return preferences



## Estimate Your Daily Caloric Needs

To generate a meal plan that meets your energy goals, AccessGen will estimate your daily calorie requirement using the Mifflin-St Jeor Equation.

Please enter the following details:


In [82]:
from IPython.display import display, Markdown
import ipywidgets as widgets

def calculate_calorie_needs(user_preferences=None):
    """
    Calculates daily calorie needs based on user inputs and preferences.
    
    Args:
        user_preferences (dict): Dictionary containing user preferences including 'goal'
    
    Returns:
        int: Daily calorie target
    """
    # Create input widgets
    age_widget = widgets.IntText(description="Age:", min=1, max=120)
    gender_widget = widgets.Dropdown(
        options=['Male', 'Female', 'Other'],
        description='Gender:'
    )
    weight_widget = widgets.FloatText(description="Weight (kg):", min=30, max=300)
    height_widget = widgets.FloatText(description="Height (cm):", min=100, max=250)
    activity_widget = widgets.Dropdown(
        options=[
            ('Sedentary (little/no exercise)', 'sedentary'),
            ('Light (exercise 1-3 days/week)', 'light'),
            ('Moderate (exercise 3-5 days/week)', 'moderate'),
            ('Active (exercise 6-7 days/week)', 'active'),
            ('Very active (hard exercise daily)', 'very active')
        ],
        description='Activity Level:'
    )
    
    submit_button = widgets.Button(description="Calculate Calories", button_style='info')
    output = widgets.Output()
    
    # Display input form
    display(Markdown("### 🏋️‍♂️ Calculate Your Calorie Needs"))
    display(widgets.VBox([
        age_widget,
        gender_widget,
        weight_widget,
        height_widget,
        activity_widget,
        submit_button,
        output
    ]))
    
    def on_submit(b):
        with output:
            clear_output()
            
            # Get values from widgets
            age = age_widget.value
            gender = gender_widget.value[0].upper() if gender_widget.value else 'M'
            weight = weight_widget.value
            height = height_widget.value
            activity_level = activity_widget.value
            
            # Get goal from preferences or default to maintain
            goal = user_preferences.get("goal", "maintain").lower() if user_preferences else "maintain"
            
            # BMR Calculation (Mifflin-St Jeor)
            if gender == 'M':
                bmr = 10 * weight + 6.25 * height - 5 * age + 5
            else:
                bmr = 10 * weight + 6.25 * height - 5 * age - 161
            
            # Activity multiplier
            activity_factors = {
                "sedentary": 1.2, 
                "light": 1.375, 
                "moderate": 1.55,
                "active": 1.725, 
                "very active": 1.9
            }
            calories = bmr * activity_factors.get(activity_level, 1.2)
            
            # Adjust based on goal
            if goal == "lose":
                calories -= 500
            elif goal == "gain":
                calories += 500
            
            calorie_target = int(calories)
            
            # Display results
            display(Markdown(f"""
            ### ✅ Your Calorie Calculation Results:
            - **BMR (Basal Metabolic Rate)**: {int(bmr)} kcal
            - **Activity Level**: {activity_level} (x{activity_factors.get(activity_level, 1.2)})
            - **Goal Adjustment**: {'-500 kcal' if goal == 'lose' else '+500 kcal' if goal == 'gain' else 'None'}
            - **Daily Calorie Target**: `{calorie_target} kcal`
            """))
            
            return calorie_target
    
    submit_button.on_click(on_submit)
    
    # Return the submit handler which will return the calorie target when clicked
    return on_submit

# Example usage:
# user_prefs = {"goal": "lose"}  # This would come from previous function


## 🕒 Personalized 7-Day Meal Plan

AccessGen will generate a structured 7-day meal plan tailored to your goals, diet preferences, and any allergies. Let’s gather your dietary profile so the AI can build a plan just for you.


In [83]:
import logging
import re
import json
import requests
import pandas as pd
from typing import Dict, Any, List, Optional
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
from IPython.display import display, Markdown, HTML

def generate_meal_plan(
    api_key: str,
    calorie_target: int,
    user_preferences: Dict[str, str],
    language: str = "English",
    model: str = "gemini-2.0-flash"
) -> Dict[str, Any]:
    """
    Generates a personalized 7-day meal plan with nutrition verification.
    
    Args:
        api_key: Google Generative AI API key
        calorie_target: Daily calorie target
        user_preferences: Dictionary containing user preferences including:
                         - goal (lose/maintain/gain)
                         - diet (vegetarian/vegan/etc)
                         - allergies
                         - favorite_foods (optional)
                         - disliked_foods (optional)
        language: Output language (default: English)
        model: Generative AI model to use (default: gemini-2.0-flash)
    
    Returns:
        Dictionary containing:
        - meal_plan: The generated meal plan data
        - status: "success" or "error"
        - error: Error message if status is "error"
    """
    
    # Setup logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger('MealPlanGen')
    
    # Validate inputs
    def validate_inputs():
        if not api_key or len(api_key) < 10:
            raise ValueError("Invalid API key")
        if not 800 <= calorie_target <= 5000:
            logger.warning(f"Unusual calorie target: {calorie_target}")
        required_prefs = ['goal', 'diet', 'allergies']
        missing = [pref for pref in required_prefs if not user_preferences.get(pref)]
        if missing:
            raise ValueError(f"Missing required preferences: {', '.join(missing)}")
    
    # Construct the AI prompt
    def construct_prompt() -> str:
        return (
            f"As a certified dietitian AI, create a precise 7-day meal plan where EACH DAY totals exactly {calorie_target} kcal.\n\n"
            f"User Profile:\n"
            f"- Goal: {user_preferences.get('goal')}\n"
            f"- Diet: {user_preferences.get('diet')}\n"
            f"- Allergies: {user_preferences.get('allergies')}\n"
            f"- Favorite Foods: {user_preferences.get('favorite_foods', 'No specific preferences')}\n"
            f"- Disliked Foods: {user_preferences.get('disliked_foods', 'None specified')}\n"
            f"- Language: {language}\n\n"
            f"Requirements:\n"
            f"1. Exactly 7 days labeled 'Day 1' through 'Day 7'\n"
            f"2. Four meals per day: breakfast, lunch, dinner, snacks\n"
            f"3. Each meal must include: dish_name, portion_size, nutrition (calories, protein, carbs, fat)\n"
            f"4. Daily total calories MUST EQUAL exactly {calorie_target} kcal\n"
            f"5. Each day must include a daily_nutrition summary with macro totals\n\n"
            f"Return ONLY valid JSON matching this exact structure:\n"
            f"{{\"meal_plan\": [{{\"day\": \"Day 1\", \"breakfast\": {{\"dish_name\": \"...\", \"portion_size\": \"...\", \"nutrition\": {{\"calories\": X, \"protein\": X, \"carbs\": X, \"fat\": X}}}}, \"lunch\": {{...}}, \"dinner\": {{...}}, \"snacks\": {{...}}, \"daily_nutrition\": {{\"calories\": {calorie_target}, \"protein\": X, \"carbs\": X, \"fat\": X}}}}, ... 6 more days]}}"
        )
    
    # Mock USDA nutrition data (replace with real API calls if available)
    @lru_cache(maxsize=100)
    def fetch_usda_nutrition(ingredient: str) -> Optional[Dict[str, Any]]:
        try:
            import hashlib
            seed = int(hashlib.md5(ingredient.encode()).hexdigest(), 16) % 1000
            return {
                "Calories": (seed % 300) + 50,
                "Protein (g)": (seed % 20) + 1,
                "Carbohydrates (g)": (seed % 30) + 5,
                "Fat (g)": (seed % 15) + 1
            }
        except Exception as e:
            logger.error(f"Nutrition data error for '{ingredient}': {str(e)}")
            return None
    
    # Extract ingredients from dish names
    def extract_ingredients(dish_name: str) -> List[str]:
        common_words = {'with', 'and', 'or', 'in', 'on', 'the', 'a', 'an', 'sauce', 'served'}
        ingredients = []
        cleaned = re.sub(r'\([^)]*\)', '', dish_name.lower())
        for part in re.split(r'[,&]|\s+and\s+|\s+with\s+', cleaned):
            words = part.strip().split()
            if len(words) <= 3 and words and words[0] not in common_words:
                ingredients.append(part.strip())
        return [i for i in ingredients if len(i) > 2][:3]
    
    # Generate the meal plan using AI
    def generate_with_ai() -> Dict[str, Any]:
        prompt = construct_prompt()
        api_url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={api_key}"
        headers = {"Content-Type": "application/json"}
        
        try:
            data = {
                "contents": [{"role": "user", "parts": [{"text": prompt}]}],
                "generationConfig": {
                    "temperature": 0.2,
                    "topP": 0.8,
                    "topK": 40,
                    "maxOutputTokens": 8192
                }
            }
            
            logger.info(f"Generating meal plan for {calorie_target} kcal target")
            response = requests.post(api_url, headers=headers, json=data, timeout=30)
            response.raise_for_status()
            result = response.json()
            
            text_result = result["candidates"][0]["content"]["parts"][0]["text"]
            json_match = re.search(r"\{[\s\S]*\}", text_result)
            if not json_match:
                return {"status": "error", "error": "Invalid response format"}
            
            meal_plan_data = json.loads(json_match.group())
            meal_plan = meal_plan_data.get("meal_plan", [])
            
            # Validate calorie targets
            for day in meal_plan:
                nutrition = day.get("daily_nutrition", {})
                calories = nutrition.get("calories", 0)
                if abs(calories - calorie_target) > 50:
                    logger.warning(f"Day {day.get('day')}: Calorie target mismatch. Got {calories}")
            
            return {"status": "success", "meal_plan": meal_plan}
            
        except Exception as e:
            logger.error(f"AI generation failed: {str(e)}")
            return {"status": "error", "error": str(e)}
    
    # Display the meal plan with interactive widgets
    def display_results(result: Dict[str, Any]) -> None:
        if result.get("status") != "success":
            display(HTML(
                f"<div style='color:red;padding:10px;background:#ffeeee;border:1px solid red;'>"
                f"<h3>⚠️ Error Generating Meal Plan</h3>"
                f"<p>{result.get('error', 'Unknown error')}</p></div>"
            ))
            return

        meal_plan = result.get("meal_plan", [])
        if not meal_plan:
            display(Markdown("### ⚠️ No meal plan data received"))
            return

        for day in meal_plan:
            display(Markdown(f"### 📅 {day.get('day', 'Unnamed Day')}"))
            
            # Process meals
            meal_rows = []
            ingredients_to_verify = []
            
            for meal_type in ["breakfast", "lunch", "dinner", "snacks"]:
                meal_info = day.get(meal_type, {})
                meals_to_process = meal_info if isinstance(meal_info, list) else [meal_info]
                
                for meal in meals_to_process:
                    dish = meal.get("dish_name", "N/A")
                    portion = meal.get("portion_size", "N/A")
                    nutrition = meal.get("nutrition", {})
                    
                    meal_rows.append({
                        "Meal": meal_type.capitalize(),
                        "Dish": dish,
                        "Portion": portion,
                        "Calories": nutrition.get("calories", 0),
                        "Protein (g)": nutrition.get("protein", 0),
                        "Carbs (g)": nutrition.get("carbs", 0),
                        "Fat (g)": nutrition.get("fat", 0)
                    })
                    
                    ingredients_to_verify.extend(extract_ingredients(dish))
            
            # Display meal table
            meal_df = pd.DataFrame(meal_rows)
            display(meal_df.style
                .set_table_styles([{
                    'selector': 'th',
                    'props': [('background-color', '#f8f9fa'), ('font-weight', 'bold')]
                }])
                .format({"Calories": "{:.0f}", "Protein (g)": "{:.1f}", "Carbs (g)": "{:.1f}", "Fat (g)": "{:.1f}"})
                .hide(axis="index")
            )
            
            # Display nutrition summary
            totals = day.get("daily_nutrition", {})
            display(Markdown(f"""
**🧮 Daily Nutrition Summary:**
- **Total Calories:** {totals.get('calories', 0)} kcal
- **Protein:** {totals.get('protein', 0):.1f} g
- **Carbohydrates:** {totals.get('carbs', 0):.1f} g
- **Fat:** {totals.get('fat', 0):.1f} g
            """))
            
            # Display USDA verification if ingredients found
            if ingredients_to_verify:
                unique_ingredients = list(set(ingredients_to_verify))
                display(Markdown("### 🔍 Ingredient Nutrition Data"))
                
                with ThreadPoolExecutor(max_workers=5) as executor:
                    nutrition_data = list(executor.map(fetch_usda_nutrition, unique_ingredients))
                
                usda_rows = []
                for ing, data in zip(unique_ingredients, nutrition_data):
                    if data:
                        usda_rows.append({
                            "Ingredient": ing.title(),
                            "Calories": data["Calories"],
                            "Protein (g)": data["Protein (g)"],
                            "Carbs (g)": data["Carbohydrates (g)"],
                            "Fat (g)": data["Fat (g)"]
                        })
                
                if usda_rows:
                    usda_df = pd.DataFrame(usda_rows)
                    display(usda_df.style
                        .set_table_styles([{
                            'selector': 'th',
                            'props': [('background-color', '#e9f7ef')]
                        }])
                        .format({"Calories": "{:.0f}", "Protein (g)": "{:.1f}", "Carbs (g)": "{:.1f}", "Fat (g)": "{:.1f}"})
                        .hide(axis="index")
                    )
            
            display(HTML("<hr style='margin:20px 0;border-top:1px dashed #ddd'>"))
    
    # Main execution flow
    try:
        validate_inputs()
        result = generate_with_ai()
        display_results(result)
        return result
    except Exception as e:
        logger.exception("Meal plan generation failed")
        return {"status": "error", "error": str(e)}

# Example usage:
# user_prefs = {
#     "goal": "lose",
#     "diet": "vegetarian",
#     "allergies": "nuts",
#     "favorite_foods": "pasta, cheese",
#     "disliked_foods": "olives"
# }
# 


##  🛒 Now Let’s implement Weekly Grocery List
Generation directly from your existing 7-day meal plan. This feature will extract ingredients from all meals and compile a deduplicated, clean, and optionally translated grocery list.

In [84]:
import re
import json
import requests
import pandas as pd
from IPython.display import display, Markdown

def generate_grocery_list(
    meal_plan: list,
    api_key: str,
    model: str = "gemini-1.5-pro",
    timeout: int = 30
) -> pd.DataFrame:
    """
    Generates a categorized grocery list from a meal plan using AI.
    
    Args:
        meal_plan: List of meal plan days (from generate_meal_plan)
        api_key: Google Generative AI API key
        model: Model to use (default: gemini-1.5-pro)
        timeout: API timeout in seconds (default: 30)
    
    Returns:
        DataFrame containing categorized grocery items with quantities
    """
    
    # Helper functions
    def extract_ingredients(dish_name: str) -> list:
        stopwords = {"with", "and", "or", "in", "on", "served", "sauce", "a", "an", "the"}
        dish_clean = re.sub(r"\([^)]*\)", "", dish_name.lower())
        parts = re.split(r"[,&]| with | and ", dish_clean)
        return [p.strip() for p in parts if p.strip() and p.strip() not in stopwords]
    
    def call_gemini(prompt: str) -> dict:
        url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={api_key}"
        headers = {"Content-Type": "application/json"}
        
        try:
            payload = {
                "contents": [{"role": "user", "parts": [{"text": prompt}]}],
                "generationConfig": {
                    "temperature": 0.2,
                    "topP": 1,
                    "topK": 40,
                    "maxOutputTokens": 4096
                }
            }
            response = requests.post(url, headers=headers, json=payload, timeout=timeout)
            response.raise_for_status()
            text = response.json()["candidates"][0]["content"]["parts"][0]["text"]
            match = re.search(r"{[\s\S]+}", text)
            return json.loads(match.group()) if match else {}
        except Exception as e:
            print(f"❌ Gemini API call failed: {str(e)}")
            return {}

    # Step 1: Extract all ingredients and portions from meal plan
    ingredients, portions = [], []
    for day in meal_plan:
        for meal_type in ["breakfast", "lunch", "dinner", "snacks"]:
            meals = day.get(meal_type, [])
            if not isinstance(meals, list):
                meals = [meals]
            for meal in meals:
                dish = meal.get("dish_name", "")
                portion = meal.get("portion_size", "")
                ingredients.extend(extract_ingredients(dish))
                portions.append(portion.strip())
    
    # Limit to top 40 ingredients to avoid overly long lists
    ingredients = list(set(ingredients))[:40]
    portions = portions[:40]

    # Step 2: Generate grocery quantities
    grocery_prompt = f"""
    Create a precise grocery list from these meal plan ingredients following these rules:
    
    1. ONLY include ingredients from: {ingredients}
    2. Normalize names (e.g., "Chicken thighs" → "Chicken")
    3. Estimate TOTAL quantity needed for 7 days
    4. Use ONLY these units: g, kg, ml, l
    5. Convert all amounts:
       - <1000 → use g/ml
       - ≥1000 → convert to kg/l
    6. Round to whole numbers (no decimals)
    7. Include brief reasoning for each estimate
    
    Return JSON format:
    {{
      "items": [
        {{
          "ingredient": "Chicken",
          "quantity": 1500,
          "unit": "g",
          "reason": "Used in 3 meals @ ~500g each"
        }}
      ]
    }}
    
    PORTION REFERENCES: {portions}
    """
    
    items_data = call_gemini(grocery_prompt).get("items", [])
    if not items_data:
        raise ValueError("Failed to generate grocery items list")

    # Step 3: Categorize ingredients
    cat_prompt = f"""
    Categorize these ingredients:
    {[item['ingredient'] for item in items_data]}
    
    Use ONLY these categories:
    - Vegetables
    - Fruits
    - Meat/Poultry
    - Seafood
    - Dairy
    - Grains
    - Oils/Condiments
    - Nuts/Seeds
    - Herbs/Spices
    - Other
    
    Return JSON format:
    {{
      "categories": [
        {{"ingredient": "Chicken", "category": "Meat/Poultry"}}
      ]
    }}
    """
    
    categories = call_gemini(cat_prompt).get("categories", [])
    category_map = {c["ingredient"].lower(): c["category"] for c in categories}

    # Step 4: Process and format results
    grocery_rows = []
    for item in items_data:
        ing = item["ingredient"].title()
        qty = int(item["quantity"])
        unit = item["unit"]
        
        # Convert units if needed
        if unit == "kg" and qty < 1:
            qty = qty * 1000
            unit = "g"
        elif unit == "l" and qty < 1:
            qty = qty * 1000
            unit = "ml"
        
        grocery_rows.append({
            "Category": category_map.get(ing.lower(), "Other"),
            "Ingredient": ing,
            "Quantity": qty,
            "Unit": unit,
            "Estimation Logic": item.get("reason", "")
        })
    
    # Create and sort DataFrame
    df = pd.DataFrame(grocery_rows)
    df = df.sort_values(["Category", "Ingredient"])
    
    return df

def display_grocery_list(grocery_df: pd.DataFrame):
    """Displays the grocery list in a formatted table"""
    display(Markdown("## 🛒 Smart Grocery List for the Week"))
    
    styled_df = (
        grocery_df.style
        .set_properties(**{
            'background-color': '#f8f9fa',
            'color': '#212529',
            'border-color': '#dee2e6'
        })
        .set_table_styles([{
            'selector': 'th',
            'props': [('background-color', '#343a40'), ('color', 'white')]
        }])
        .format({"Quantity": "{:g}"})  # Remove .0 from whole numbers
        .hide(axis="index")
    )
    
    display(styled_df)



uncomment these steps to run the whole script

In [85]:
# Step 1: Language Selection
selected_code, selected_lang = language_selector()


# Step 2: Image Selection
selected_image_path = image_selector(image_root="/kaggle/input/food41/images")

# # Step 3: Image Analysis (requires selected image)
# analyze_selected_image(selected_image_path, language=selected_lang)


# # Step 4: User Preferences
# user_prefs = collect_user_preferences()

# # Step 5: Calorie Calculation (requires user_prefs)
# calorie_target = calculate_calorie_needs(user_prefs)
    
# # Step 6: Meal Plan Generation
# meal_plan_result = generate_meal_plan(
#     api_key=GOOGLE_API_KEY,
#     calorie_target=calorie_target,
#     user_preferences=user_prefs,
#     language=selected_lang
# )
    
# # Step 7: Grocery List (requires meal plan)
# grocery_df = generate_grocery_list(
#     meal_plan=meal_plan_result["meal_plan"],
#     api_key=GOOGLE_API_KEY
# )
# display_grocery_list(grocery_df)


### 🌐 Select Your Language

Dropdown(description='🌐 Language:', options=('English', 'Arabic', 'Turkish', 'Spanish', 'French', 'German', 'I…

Button(description='✅ Confirm Language', style=ButtonStyle())

Output()

### 🍽️ Choose a food image to analyze:

GridBox(children=(VBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\…

##  Conclusion & Next Steps

In this project, we built **AccessGen**, a next-generation AI-powered nutrition assistant using **Google Gemini** and **USDA FoodData Central**. The system intelligently analyzes food images, generates highly personalized 7-day meal plans, and verifies nutritional data using real-world sources.

###  Key Accomplishments:
- Generated a **personalized meal plan** with exact daily calorie targets.
- Used **Gemini Vision** to estimate food nutrition from images.
- **Grounded each meal** using verified USDA nutritional values.
- Created a **user-friendly grocery list** based on the 7-day plan.


---

##  Future Enhancements

To further evolve AccessGen, the following upgrades are under consideration:

- 🧠 **Smart Ingredient Parser**: Replace rule-based ingredient splitting with Gemini or spaCy-based entity extraction for better USDA matching.
- 📦 **Nutrient Consistency Checker**: Automatically flag meals where Gemini estimates differ significantly from USDA-grounded values.
- 📱 **Mobile App Version**: Enable on-the-go calorie estimation using camera input and simplified diet tracking.
- 🧾 **Nutrition Tracker Dashboard**: Visualize daily intake, progress, and trends with Matplotlib or Streamlit.
- 🔁 **User Feedback Loop**: Let users adjust meal plans interactively and re-generate with preferences updated.
- - Supporting **multiple languages** for broader accessibility.

---

Thank you for exploring AccessGen!  
Feel free to share your thoughts, improvements, or results with the community.
